### Define Inception Model

In [ ]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, BatchNormalization
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

img_width, img_height = 320, 240
train_data_dir = 'datasets/UCF11/frames/train'
validation_data_dir = 'datasets/UCF11/frames/test'
nb_train_samples = 7365
nb_validation_samples = 1825 
batch_size = 64
epochs = 50
nb_class = 11

model = applications.inception_v3.InceptionV3(weights="imagenet", include_top=False, input_shape=(img_width, img_height, 3))

### Use Inception as Backbone

In [ ]:
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(nb_class, activation="softmax")(x)

model_final = Model(inputs=model.input, outputs=predictions)

### Convert BatchNorm to LocalNorm

In [ ]:
from scripts.layers.LocalNorm import LocalNormalization

group_size = 16

for i in range(len(model_final.layers)):
    if type(model_final.layers[i]) == BatchNormalization:
        model_final.layers[i] = LocalNormalization(groupsize=group_size, batch_size=batch_size)

### Freeze Layers

In [ ]:
for layer in model_final.layers[-5:]:
    if type(model_final.layers[i]) != LocalNormalization:
        layer.trainable = False

### Data Generators

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    horizontal_flip=True,
    rotation_range=10.,
    width_shift_range=0.2,
    height_shift_range=0.2
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size, 
    class_mode="categorical"
)

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    class_mode="categorical"
)

### Train Model

In [ ]:
model_final.compile(loss="categorical_crossentropy", optimizer=optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])

checkpoint = ModelCheckpoint("inception_v3_1.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

model_final.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks=[checkpoint, early]
)